# Database Setup

Let's walk through an option for coffee market analysis:

<br>[Gather our data](#gather-data)
<br>[Setup our database](#setup-database)
<br>[Import ICO data](#import-data)

## Gather our data <a class="anchor" id="gather-data"></a>

There are many sites that offer coffee statistics, but if we really want to dig into the details of how coffee flows along the supply chain, we will need to get our hands on some detailed data points.

The International Coffee Organization (ICO) is "the main intergovernmental organization for coffee, bringing together exporting and importing Governments to tackle the challenges facing the world coffee sector through international cooperation. Its Member Governments represent 98% of world coffee production and 67% of world consumption."<sup>1</sup>  Although ICO data does not comprehensively cover all coffee movements, it is the single greatest source of data regarding yearly coffee movements around the globe.  Let's start there.

---
Let's go to the ICO site and find the "Historical Data" section.
![ICO Historical Data](img/ICO_historical_data.png)
<br>
<br>
At first glance, it can be hard to understand how the data is organized.
![ICO Historical Data](img/ICO_historical_data2.png)
<br>
<br>
We can easily see which data is gathered on a calendar basis vs. a seasonal basis ("crop year"), but it can be hard to initially understand which sub-sections refer to exporters vs. importers.  We will separate our data into "seasonal" and "calendar" data and the five types of commodity movements: Inventory + Production + Imports - Exports - Consumption.  We will also focus mostly on the "SOURCE" data, since this is originally reported data and any "CALC" data we should be able to calculate on our own.  We will include some imported consumption data (calculated) for convenience:

```
Description                                 |File| Origin |Movement| Database Name
--------------------------------------------------------------------------------------------
Supply Data
- Total production - Crop Year              | 1A | SOURCE | export | seasonal_production
- Domestic consumption - Crop Year          | 1B | SOURCE | export | seasonal_consumption
- Exportable production - Crop Year         | 1C | CALC   | export | 
- Gross opening stocks - Crop Year          | 1D | SOURCE | export | seasonal_inventory
- Exports - Crop Year                       | 1E | SOURCE | export | seasonal_exports

Trade Statistics Data
- Exports - Calendar Year                   | 2A | SOURCE | export | calendar_exports
- Imports - Calendar Year                   | 2B | SOURCE | import | calendar_imports
- Re-exports - Calendar Year                | 2C | SOURCE | import | calendar_exports

Inventories/Consumption Data
- Inventories - End of Year                 | 4A | SOURCE | import | calendar_inventory
- Disappearance (consumption) - End of Year | 4B | CALC   | import | calendar_consumption

Non-Member Data
- Imports - Calendar Year                   | 5A | SOURCE | import | calendar_imports
- Re-exports - Calendar Year                | 5B | SOURCE | import | calendar_exports
```

<br><sup>1</sup> http://www.ico.org/mission07_e.asp?section=About_Us

## Setup our database <a class="anchor" id="setup-database"></a>

Before we can do any analysis we will need to set up our database.  We will use a PostgreSQL database on macOS.

Please [install PostgreSQL](https://www.postgresql.org/) if needed.

Start the postgresql service, create the `coffee` database, and enter the postgres environment:
```
brew services start postgresql
createdb coffee
psql coffee
```
Create the database admin and add privileges:
```
CREATE USER coffeeadmin WITH PASSWORD [password];
GRANT ALL PRIVILEGES ON DATABASE coffee TO coffeeadmin;
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO coffeeadmin;
```
We will need to convert text months and types of coffee to standardized IDs, so let's go ahead and manually create and fill these tables:
```
CREATE TABLE month (id int PRIMARY KEY, title text);
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO coffeeadmin;
INSERT INTO month (id, title) VALUES (1,'January');
INSERT INTO month (id, title) VALUES (2,'February');
INSERT INTO month (id, title) VALUES (3,'March');
INSERT INTO month (id, title) VALUES (4,'April');
INSERT INTO month (id, title) VALUES (5,'May');
INSERT INTO month (id, title) VALUES (6,'June');
INSERT INTO month (id, title) VALUES (7,'July');
INSERT INTO month (id, title) VALUES (8,'August');
INSERT INTO month (id, title) VALUES (9,'September');
INSERT INTO month (id, title) VALUES (10,'October');
INSERT INTO month (id, title) VALUES (11,'November');
INSERT INTO month (id, title) VALUES (12,'December');
```
```
CREATE TABLE coffee_type (id int, title text, symbol text);
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO coffeeadmin;
INSERT INTO coffee_type (id, title, symbol) VALUES (1,'Arabica','(A)');
INSERT INTO coffee_type (id, title, symbol) VALUES (2,'Robusta','(R)');
INSERT INTO coffee_type (id, title, symbol) VALUES (3,'Both','(A/R)');
INSERT INTO coffee_type (id, title, symbol) VALUES (3,'Both','(R/A)');
```
The ICO location names are unique, but we probably want to create some tables that allow each location to have an ID and be compatible with a variety of text variations, due to language and other spelling differences we might encounter:
```
CREATE TABLE location (id int PRIMARY KEY, title text, parent_location_id int);
CREATE TABLE location_name (location_id int, name text);
```
In the top directory is a `locations.xlsx` file I compiled with standardized location names, assigned location IDs, parent location IDs (allows us to nest locations for grouped analysis), and a list of possible variations on the location name which also include all name versions used by the ICO data.  This allows us to standardize all locations and even combine this data with other data sources later, if needed.  Let's write a Python script to pull the locations out of this Excel file and add them to the database.

Remember to grant our admin account access to the new tables:
```
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO coffeeadmin;
```

The script to import the locations and location names is relatively simple. . .

In [ ]:
wb = open_workbook('locations.xlsx')
    for s in wb.sheets():
        print('Sheet: %s' % (s.name))
        if s.name == 'ALL':
            for row in range(1, s.nrows):
                id = int(s.cell(row, 0).value)
                parent = int(s.cell(row, 1).value)
                location = s.cell(row, 2).value
                names = tuple(s.cell(row, 3).value.split('|'))
                query = ("INSERT INTO location"
                         " (id,parent_location_id,title)"
                         " VALUES (%s, %s, %s)"
                         " RETURNING id")
                sql(query, id, parent, location)
                for name in names:
                    query = ("INSERT INTO location_name"
                             " (location_id,name)"
                             " VALUES (%s, %s)"
                             " RETURNING location_id")
                    sql(query, id, name)

. . .but we will need some support methods, so let's just execute it from the `utils` package:

```
MacBook-Air:src seanhart$ python3
Python 3.7.5 (default, Nov  1 2019, 02:16:32) 
[Clang 11.0.0 (clang-1100.0.33.8)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> 
>>> import utils
>>> utils.locations_to_db()
Sheet: ALL
Sheet: STANDARD
>>> 
>>> exit()
```

Now let's check that all of the locations were processed.  We should have 206 locations and almost 250 variations on location names:

```
psql coffee
coffee=# SELECT COUNT(*) FROM location;
 count 
-------
   206
(1 row)

coffee=# 
coffee=# 
coffee=# SELECT COUNT(*) FROM location_name;
 count 
-------
   248
(1 row)
```

Ok, that looks correct.  Let's move on to importing the supply chain data.

## Import ICO data <a class="anchor" id="import-data"></a>

We need to extract the data from the ICO Excel files.

We will need to create tables for each of the major movement types (inventory, production, imports, exports, consumption) for both seasonal and calendar data.  Seasonal data includes the harvest month and coffee type so let's add those columns in case we want to distinguish via those data points later:
```
CREATE TABLE seasonal_inventory (id text PRIMARY KEY, year int, harvest_month_id int, location_id int, coffee_type int, value float);
CREATE TABLE seasonal_production (id text PRIMARY KEY, year int, harvest_month_id int, location_id int, coffee_type int, value float);
CREATE TABLE seasonal_exports (id text PRIMARY KEY, year int, harvest_month_id int, location_id int, coffee_type int, value float);
CREATE TABLE seasonal_consumption (id text PRIMARY KEY, year int, harvest_month_id int, location_id int, coffee_type int, value float);

CREATE TABLE calendar_inventory (id text PRIMARY KEY, year int, location_id int, value float);
CREATE TABLE calendar_imports (id text PRIMARY KEY, year int, location_id int, value float);
CREATE TABLE calendar_exports (id text PRIMARY KEY, year int, location_id int, value float);
CREATE TABLE calendar_consumption (id text PRIMARY KEY, year int, location_id int, value float);
```
Remember to grant our admin account access to the new tables:
```
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA public TO coffeeadmin;
```

Now let's create and run the scripts to import the Excel data from ICO.  We will access the Excel files from the links directly so we can simply call the scripts in the future when we want to update the data.

(Note that the scripts call a local `utils` file that contains some commonly used methods including the basic SQL query method that will access our database using the `psycopg2` library.)

Here are the scripts we will use (in the `/data_parse` directory), but skip below and we will run them from the terminal for ease.

### ICO Seasonal Data Update

In [ ]:
from .. import utils


# Excel sheet links
link_exporters_inventory = 'http://www.ico.org/historical/1990%20onwards/Excel/1d%20-%20Gross%20Opening%20stocks.xlsx'
link_exporters_production = 'http://www.ico.org/historical/1990%20onwards/Excel/1a%20-%20Total%20production.xlsx'
link_exporters_exports = 'http://www.ico.org/historical/1990%20onwards/Excel/1e%20-%20Exports%20-%20crop%20year.xlsx'
link_exporters_consumption = 'http://www.ico.org/historical/1990%20onwards/Excel/1b%20-%20Domestic%20consumption.xlsx'

# Get the month and coffee types conversions
month_dict = utils.get_months()
coffee_type_dict = utils.get_coffee_types()

# The first column will be the location names
# The second column will be coffee type (1=A,2=R,3=Both)
# The third column will start the harvest data
location_col = 0
coffee_type_col = 1
value_col = 2

# Store the data in a local dict to sum any split locations
# data is stored in id:value format
data = {}


def seasonal_data_update(link, sheet_name, data_header, db_name):
    # Reset the dict
    global data
    data = {}

    # Get the workbook from the link
    wb = utils.get_workbook_at(link)

    # Set the default harvest month
    harvest_month_id = 1

    # Find the correct sheet in the workbook
    sheet = find_sheet(wb, sheet_name)

    # Find the first row with the passed keyword in the first column
    topRow = find_data_start_row(sheet, 0, data_header)

    for col in range(value_col, sheet.ncols):
        # First get the crop year - it will be two years,
        # so grab the first two numbers for century
        # and last two numbers for year
        year = int(sheet.cell(topRow, col).value[0:2] + sheet.cell(topRow, col).value[5:7])
        # The 1999/00 season will be translated to the year 1900
        # correct this to the year 2000
        if year == 1900:
            year = 2000

        # Grab the location data on each row for this crop year
        for row in range(topRow+1, sheet.nrows):

            # If the location cell is empty it is not a countable cell
            if sheet.cell(row, location_col).value != "":

                # If the location cell has the word 'group' in the text
                # update the harvet_month to the latest month
                group_text_start = sheet.cell(row, location_col).value.find('group')
                if group_text_start != -1:
                    harvest_month_text = sheet.cell(row, location_col).value[0:group_text_start - 1]
                    harvest_month_id = month_dict[harvest_month_text]
                    continue

                # If you get to the 'Total' row, stop
                if sheet.cell(row, location_col).value == 'Total':
                    break

                # If the first column might have two spaces at the beginning
                # of the cell - be sure to clean up the text
                location_name = sheet.cell(row, location_col).value
                location_name = location_name.strip()
                # print(location_name)

                # Find the location id based off of the used name
                location_result = utils.sql("SELECT location_id FROM location_name WHERE name=%s", location_name)
                # Raise error and skip if the location cannot be found
                if len(location_result) < 1:
                    print("ERROR - LOCATION '%s' NOT FOUND" % (location_name))
                    continue

                # If the results have more than one tuple, we have duplicate location names
                if len(location_result[0]) != 1:
                    print("ERROR - LOCATION '%s' ENTRY ERROR" % (location_name))
                    continue

                location_id = location_result[0][0]
                # print(location_id)
                id = '%s-%d-%d' % (location_id, year, harvest_month_id)

                # Check to ensure the value is not empty, if so assign 0
                value = float(0)
                if sheet.cell(row, col).value != '':
                    value = float(sheet.cell(row, col).value)

                # Get the value to use after checking for split locations
                value = calculate_location_total(id, value)

                # Translate the coffee type
                coffee_type = coffee_type_dict[sheet.cell(row, coffee_type_col).value]

                # print('%s, %d, %d, %d, %d, %.4f' % (id, year, harvest_month_id, location_id, coffee_type, value))
                insert = ("INSERT INTO " + db_name +
                          " (id,year,harvest_month_id,location_id,coffee_type,value)"
                          " VALUES (%s, %s, %s, %s, %s, %s)"
                          " ON CONFLICT ON CONSTRAINT " + db_name + "_pkey"
                          " DO UPDATE SET coffee_type=%s,value=%s"
                          " RETURNING id")
                utils.sql(insert, id, year, harvest_month_id, location_id, coffee_type, value, coffee_type, value)


# If a location has been split over several location names
# the values will need to be summed
def calculate_location_total(id, value):
    # Check whether the id already exists - if it does,
    # sum the previous value with the new value and
    # return the new total value for overwriting the old value
    new_value = 0
    if id in data:
        print('%s: %f' % (id, value))
        new_value = data[id] + value
    else:
        new_value = value

    # Record the entry in the data dict
    data[id] = new_value
    return new_value


def find_data_start_row(sheet, column, text):
    # Find the first row in the passed column with the passed text
    for row in range(sheet.nrows):
        if sheet.cell(row, column).value == text:
            return row


def find_sheet(workbook, sheet_name):
    for s in workbook.sheets():
        print('Sheet: %s' % (s.name))
        if s.name == sheet_name:
            return s


if __name__ == "__main__":
    print("ICO seasonal data update")
    seasonal_data_update(link_exporters_inventory, 'Print Table here', 'Crop years', 'seasonal_inventory')
    seasonal_data_update(link_exporters_production, 'Production', 'Crop year', 'seasonal_production')
    seasonal_data_update(link_exporters_exports, 'Exports', 'Crop year', 'seasonal_exports')
    seasonal_data_update(link_exporters_consumption, 'Print Table here', 'Crop year', 'seasonal_consumption')


### ICO Calendar Year Data Update

In [ ]:
from .. import utils


# Excel sheet links
link_importers_inventory = 'http://www.ico.org/historical/1990%20onwards/Excel/4a%20-%20Inventories.xlsx'
link_importers_imports = 'http://www.ico.org/historical/1990%20onwards/Excel/2b%20-%20Imports.xlsx'
link_importers_other_imports = 'http://www.ico.org/historical/1990%20onwards/Excel/5a%20-%20Non-member%20imports.xlsx'
link_importers_exports = 'http://www.ico.org/historical/1990%20onwards/Excel/2c%20-%20Re-exports.xlsx'
link_importers_other_exports = 'http://www.ico.org/historical/1990%20onwards/Excel/5b%20-%20Non-member%20re-exports.xlsx'
link_importers_consumption = 'http://www.ico.org/historical/1990%20onwards/Excel/4b%20-%20Disappearance.xlsx'

# The first column will be the location names
# The second column will start the data
location_col = 0
value_col = 1

# Store the data in a local dict to sum any split locations
# data is stored in id:value format
data = {}


def calendar_data_update(link, sheet_name, db_name):
    # Reset the dict
    global data
    data = {}

    # Get the workbook from the link
    wb = utils.get_workbook_at(link)

    # Find the correct sheet in the workbook
    sheet = find_sheet(wb, sheet_name)

    # Find the first row that is not blank
    topRow = find_data_start_row(sheet, 1)

    for col in range(value_col, sheet.ncols):
        # First get the calendar year - it will be two years,
        # so grab the first two numbers for century
        # and last two numbers for year
        year = int(sheet.cell(topRow, col).value)

        # Grab the location data on each row for this year
        # skip the header row
        for row in range(topRow+1, sheet.nrows):

            # If the location cell contains the word 'inventories'
            # or is empty it is not a countable cell
            inventories_text_start = sheet.cell(row, location_col).value.find('Inventories')
            if inventories_text_start == -1 and sheet.cell(row, location_col).value != "":

                # If you get to the 'Total' row, stop
                if sheet.cell(row, location_col).value == 'Total':
                    break

                # If the first column might have two spaces at the beginning
                # of the cell - be sure to clean up the text
                location_name = sheet.cell(row, location_col).value
                location_name = location_name.strip()
                # print(location_name)

                # Find the location id based off of the used name
                location_result = utils.sql("SELECT location_id FROM location_name WHERE name=%s", location_name)
                # Raise error and skip if the location cannot be found
                if len(location_result) < 1:
                    print("ERROR - LOCATION '%s' NOT FOUND" % (location_name))
                    continue

                # If the results have more than one tuple, we have duplicate location names
                if len(location_result[0]) != 1:
                    print("ERROR - LOCATION '%s' ENTRY ERROR" % (location_name))
                    continue

                location_id = location_result[0][0]
                # print(location_id)
                id = '%d-%d' % (location_id, year)

                # Check to ensure the value is not empty, if so assign 0
                value = float(0)
                if sheet.cell(row, col).value != '':
                    value = float(sheet.cell(row, col).value)

                # Get the value to use after checking for split locations
                value = calculate_location_total(id, value)

                # print("%s, %d, %d, %d" % (id, year, location_id, value))
                insert = ("INSERT INTO " + db_name +
                          " (id,year,location_id,value)"
                          " VALUES (%s, %s, %s, %s)"
                          " ON CONFLICT ON CONSTRAINT " + db_name + "_pkey"
                          " DO UPDATE SET value=%s"
                          " RETURNING id")
                utils.sql(insert, id, year, location_id, value, value)


# If a location has been split over several location names
# the values will need to be summed
def calculate_location_total(id, value):
    # Check whether the id already exists - if it does,
    # sum the previous value with the new value and
    # return the new total value for overwriting the old value
    new_value = 0
    if id in data:
        print('%s: %f' % (id, value))
        new_value = data[id] + value
    else:
        new_value = value

    # Record the entry in the data dict
    data[id] = new_value
    return new_value


def find_data_start_row(sheet, column):
    # Find the first row in the passed column with the passed text
    for row in range(sheet.nrows):
        if sheet.cell(row, column).value != '':
            return row


def find_sheet(workbook, sheet_name):
    for s in workbook.sheets():
        print('Sheet: %s' % (s.name))
        if s.name == sheet_name:
            return s


if __name__ == "__main__":
    print("ICO calendar year data update")
    calendar_data_update(link_importers_inventory, 'Inventories', 'calendar_inventory')
    calendar_data_update(link_importers_imports, 'Imports', 'calendar_imports')
    calendar_data_update(link_importers_other_imports, 'Imports', 'calendar_imports')
    calendar_data_update(link_importers_exports, 'Re-exports', 'calendar_exports')
    calendar_data_update(link_importers_other_exports, 'Re-exports', 'calendar_exports')
    calendar_data_update(link_importers_consumption, 'Disappearance', 'calendar_consumption')


Run the calendar data import script (using the `-m` flag to use the Python module namespace) from the package root directory:
```
MacBook-Air:coffee seanhart$ python3 -m src.data_parse.calendar
```

Now let's check our database and see if the data was imported correctly:
```
MacBook-Air:coffee seanhart$ psql coffee
psql (12.1)
Type "help" for help.

coffee=# \d
                List of relations
 Schema |         Name         | Type  |  Owner   
--------+----------------------+-------+----------
 public | calendar_consumption | table | seanhart
 public | calendar_exports     | table | seanhart
 public | calendar_imports     | table | seanhart
 public | calendar_inventory   | table | seanhart
 public | coffee_type          | table | seanhart
 public | location             | table | seanhart
 public | location_name        | table | seanhart
 public | month                | table | seanhart
 public | seasonal_consumption | table | seanhart
 public | seasonal_exports     | table | seanhart
 public | seasonal_inventory   | table | seanhart
 public | seasonal_production  | table | seanhart
(12 rows)

coffee=# SELECT * FROM calendar_consumption WHERE year=2018 ORDER BY location_id DESC LIMIT 5;
    id    | year | location_id | value 
----------+------+-------------+-------
 100-2018 | 2018 |         100 |    29
 99-2018  | 2018 |          99 |    87
 98-2018  | 2018 |          98 |   715
 97-2018  | 2018 |          97 |     0
 96-2018  | 2018 |          96 |     0
(5 rows)
```

We can run the same test but join with the location table to show the location titles:
```
coffee=# SELECT lid.title AS location,pid.title AS parent,cons.year,cons.value AS consumption
coffee-# FROM calendar_consumption cons
coffee-# INNER JOIN location lid ON cons.location_id=lid.id
coffee-# INNER JOIN location pid ON lid.parent_location_id=pid.id
coffee-# WHERE year=2018 ORDER BY location_id ASC LIMIT 5;
  location   |   parent   | year |    consumption     
-------------+------------+------+--------------------
 EU          | Europe     | 2018 |       44729.571005
 Japan       | Asia-North | 2018 |       7833.5956675
 Norway      | Europe     | 2018 |        734.1667393
 Russia      | Asia-North | 2018 |       4233.5037736
 Switzerland | Europe     | 2018 | 1170.2869719999999
(5 rows)
```

We can run similar test queries on all the tables and see the data appears to be formatted correctly, and if we double-check some random rows with the Excel file the data appears to be accurate.  For additional checking, we can also run tests on the totals for each table (by year) and compare to the Excel file, if desired:

(Please note that imports and exports include both the regular imports/exports files and the "other" imports/exports files, so each database table total will include the sum of both file totals.)
```
coffee=# SELECT year,SUM(value) FROM calendar_consumption GROUP BY year ORDER BY year;
 year |        sum         
------+--------------------
 1990 |  93551.86161529997
 1991 | 101593.80189780003
 1992 | 103121.07931870001
 1993 | 108995.99266259998
 1994 | 100591.84517909998
 1995 | 104327.99943299996
 1996 | 106821.23959110002
 1997 | 105876.65767690002
 1998 | 107891.48143409999
 1999 | 107384.10738219999
 2000 | 105611.71093510001
 2001 |     108461.2095811
 2002 | 109274.60534609997
 2003 | 112721.15981410001
 2004 |     116346.4352132
 2005 | 113095.83543549998
 2006 |     116182.3202563
 2007 |     116589.1928358
 2008 | 115930.80449770001
 2009 |     113732.1700961
 2010 |     117126.2707015
 2011 | 116561.26319139998
 2012 |      117207.713997
 2013 |             119985
 2014 |      80296.5465616
 2015 |      80318.9926862
 2016 |      84482.4657011
 2017 |      82270.6014287
 2018 |      85748.2519519
(29 rows)
```

You made it!  Your database is ready for analysis.

<br>[Home](coffee.ipynb)
<br>[Producer Analysis](producers.ipynb)
<br>[Consumer Analysis](consumers.ipynb)